# Load and transform a very large text file

Requires a folder called 'csv' to store the results

In [9]:
import pandas as pd
import numpy as np
import os
import ast
import glob
from datetime import datetime

In [10]:
def extract_data_large (rawdata):        
    
        dict_ = {"skills":[],"skills2":[], "rate":[], "amount":[], "job_type":[], "date":[], "description":[],
         "title":[], "worker_ID":[], "op_engagement":[], "engagement_weeks":[],
         "op_pref_english_skill":[],"candidates":[], "buyer":[],"category_1":[],"category_2":[],
        "assignments":[],"assignment_info":[]} 
    
    
        job = ast.literal_eval(rawdata)

        # multiple items

        ## skills 1

        try:
            skills = list(job["op_required_skills"]["op_required_skill"].values())[0]
        except:
            skills = float('nan')
        dict_["skills"].append(skills)

        ## skills multiple
        try:
            skills2 = job["op_required_skills"]["op_required_skill"]
            skill_list = []
            for skill in skills2:
                try:
                    skill_item = list(skill.values())[0]
                except:
                    skill_item = float('nan')
                skill_list.append(skill_item)
        except:
            skill_list = float('nan')

        dict_["skills2"].append(skill_list)

        ## candidates
        try:
            candidates = list(job["candidates"].values())[0]

            candidate_list = []
            for candidate in candidates:
                try:
                    candidate_item = candidate
                except:
                    candidate_item = float('nan')
                candidate_list.append(candidate_item)
        except:
            candidate_list = float('nan')

        dict_["candidates"].append(candidate_list)

        # Simple items

        try:
            assignments = job["assignments"]
        except:
            assignments = float('nan')
        dict_["assignments"].append(assignments)    

        try:
            assignment_info = job["assignment_info"]
        except:
            assignment_info = float('nan')
        dict_["assignment_info"].append(assignment_info)    

        # project category sub-group
        try:
            category_2 = list(job["op_job_category_v2"].values())[0]["name"]
        except:
            category_2 = float('nan')
        dict_["category_2"].append(category_2) 

        # project category main group
        try:
            category_1 = list(list(job["op_job_category_v2"].values())[0]["groups"].values())[0]['name']
        except:
            category_1 = float('nan')
        dict_["category_1"].append(category_1) 

        try:
            buyer = job["buyer"]
        except:
            buyer = float('nan')
        dict_["buyer"].append(buyer) 

        try:
            title = job["op_title"]
        except:
            title = float('nan')
        dict_["title"].append(title)    

        try:
            description = job["op_description"]
        except:
            description = float('nan')
        dict_["description"].append(description)

        try:
            worker_ID = job["ciphertext"]
        except:
            worker_ID = float('nan')
        dict_["worker_ID"].append(worker_ID)

        try:
            op_engagement = job["op_engagement"]
        except:
            op_engagement = float('nan')
        dict_["op_engagement"].append(op_engagement)

        try:
            engagement_weeks = job["engagement_weeks"]
        except:
            engagement_weeks = float('nan')
        dict_["engagement_weeks"].append(engagement_weeks)   

        try:
            op_pref_english_skill = job["op_pref_english_skill"]
        except:
            op_pref_english_skill = float('nan')
        dict_["op_pref_english_skill"].append(op_pref_english_skill)    

        try:
            date = job["op_ctime"]
        except:
            date = float('nan')
        dict_["date"].append(date)

        try:    
            job_type =job["op_job_category_v2"]["op_job_category_v"]["name"]
        except:
            job_type = float('nan')
        dict_["job_type"].append(job_type)

        if job["job_type"] == "Hourly":

            try:
                rate = (int(job["op_high_hourly_rate_all"]) + int(job["op_low_hourly_rate_all"])) *0.5
                amount = float('nan')
            except:
                rate = float('nan')
        else:
            try:
                amount = job["amount"]
            except:
                amount = float('nan')
            rate = float('nan')
        
        try:
            dict_["rate"].append(rate)
            dict_["amount"].append(amount)
            
        except:
            dict_["rate"].append(float('nan'))
            dict_["amount"].append(float('nan'))
        
        return dict_

In [11]:
# Transform to dataframe and clean

def to_dataframe(dict_):
    
    df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in dict_.items()]))
    
    # clean skills
    df.skills.fillna(df.skills2, inplace=True)
    del df['skills2']
    
    # Clean assignments and assignment_info columns
    df = df.replace(r'^\s*$', np.nan, regex=True)
    
    return df

In [18]:
def clean_df(df):
    
    # limit to only hourly projects
    df.rate = df.rate.fillna(-1)
    fixed = df[ df['rate'] == -1].index
    df.drop(fixed, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Transform date (delete the last three zeros)
    df.date = df.date.astype(int)
    df.date = (df.date/1000).astype(int)
    df['date_2'] = df.date.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
    
    # Extract buyer information

    # Country of employer
    country = []

    for i in df.buyer.index:
        try:
            country.append(df.buyer[i]['op_country'])
        except:
            country.append(np.nan)

    df['employer_country'] = country

    # city of employer
    city = []

    for i in df.buyer.index:
        try:
            city.append(df.buyer[i]['op_city'])
        except:
            city.append(np.nan)

    df['employer_city'] = city
    
    # drop assignment and candidate information
    df.drop(columns=['candidates','assignments','assignment_info','buyer','amount'], inplace=True)
    
    return df

# Execution

In [19]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
df11 = pd.DataFrame()

with open("results_4_to_10.txt") as f:
    
    for number,line in enumerate(f):
        
        # chunk 1
        if number < 15000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df1 = df1.append(df)
        
        elif number == 15000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df1 = df1.append(df)
            df1.to_csv("csv/" + "chunk1"+".csv", index= False)
            del df1
            print(number)
            
        # chunk 2
        elif 15000 < number < 30000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df2 = df2.append(df)
        
        elif number == 30000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df2 = df2.append(df)
            df2.to_csv("csv/" + "chunk2"+".csv", index= False)
            del df2
            print(number)
            
        # chunk 3
        elif 30000 < number < 45000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df3 = df3.append(df)
        
        elif number == 45000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df3 = df3.append(df)
            df3.to_csv("csv/" + "chunk3"+".csv", index= False)
            del df3
            print(number)
            
        # chunk 4
        elif 45000 < number < 60000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df4 = df4.append(df)
        
        elif number == 60000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df4 = df4.append(df)
            df4.to_csv("csv/" + "chunk4"+".csv", index= False)
            del df4
            print(number)

        # chunk 5
        elif 60000 < number < 75000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df5 = df5.append(df)
        
        elif number == 75000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df5 = df5.append(df)
            df5.to_csv("csv/" + "chunk5"+".csv", index= False)
            del df5
            print(number)
            
        # chunk 5
        elif 60000 < number < 75000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df5 = df5.append(df)
        
        elif number == 75000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df5 = df5.append(df)
            df5.to_csv("csv/" + "chunk5"+".csv", index= False)
            del df5
            print(number)
        
        # chunk 6
        elif 75000 < number < 90000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df6 = df6.append(df)
        
        elif number == 90000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df6 = df6.append(df)
            df6.to_csv("csv/" + "chunk6"+".csv", index= False)
            del df6
            print(number)

        # chunk 7
        elif 90000 < number < 105000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df7 = df7.append(df)
        
        elif number == 105000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df7 = df7.append(df)
            df7.to_csv("csv/" + "chunk7"+".csv", index= False)
            del df7
            print(number)
            
        # chunk 8
        elif 105000 < number < 120000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df8 = df8.append(df)
        
        elif number == 120000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df8 = df8.append(df)
            df8.to_csv("csv/" + "chunk8"+".csv", index= False)
            del df8
            print(number)
            
        # chunk 9
        elif 120000 < number < 135000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df9 = df9.append(df)
        
        elif number == 135000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df9 = df9.append(df)
            df9.to_csv("csv/" + "chunk9"+".csv", index= False)
            del df9
            print(number)

        # chunk 10
        elif 135000 < number < 150000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df10 = df10.append(df)
        
        elif number == 150000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df10 = df10.append(df)
            df10.to_csv("csv/" + "chunk10"+".csv", index= False)
            del df10
            print(number)     
        
        # chunk 11
        else:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df11 = df11.append(df)
            df11.to_csv("csv/" + "chunk11"+".csv", index= False)
            del df11
            print(number) 

15000
30000
45000
60000
75000
90000
105000
120000
135000
150000
150001


NameError: name 'df11' is not defined

# Second Half

In [24]:
df11 = pd.DataFrame()
df12 = pd.DataFrame()
df13 = pd.DataFrame()


with open("results_4_to_10.txt") as f:
    
    for number,line in enumerate(f):
        
        if number < 150000:
            pass
        
        # chunk 11
        elif 150000 < number < 165000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df11 = df11.append(df)
        
        elif number == 165000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df11 = df11.append(df)
            df11.to_csv("csv/" + "chunk11"+".csv", index= False)
            del df11
            print(number)
            
        # chunk 12
        elif 165000 < number < 180000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df12 = df12.append(df)
        
        elif number == 180000:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df12 = df12.append(df)
            df12.to_csv("csv/" + "chunk12"+".csv", index= False)
            del df12
            print(number)
            
        # chunk 13
        elif 180000 < number < 184146:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df13 = df13.append(df)
        
        elif number == 184146:
            df = clean_df(to_dataframe(extract_data_large(line)))
            df13 = df13.append(df)
            df13.to_csv("csv/" + "chunk13"+".csv", index= False)
            #del df13
            print(number)

        else:
            pass

165000
180000
